In [642]:
import polars as pl
import numpy as np
import plotly.express as px

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')
pl.Config.set_tbl_rows(1000)


polars.config.Config

In [643]:
SEED = 100622
TARGET = 'Salary'
ID = 'id'
N_FOLDS = 5

## Data read

In [644]:
def load_data(data_path="../data/"):
    """Load all required datasets."""
    df_salary = pl.read_csv(f"{data_path}salary.csv")
    df_people = pl.read_csv(f"{data_path}people.csv")
    df_descriptions = pl.read_csv(f"{data_path}descriptions.csv")
    return df_salary, df_people, df_descriptions

In [645]:
# prompt: fix the error below by adding drive to the environment

if 'google.colab' in str(get_ipython()):
    # Mount Google Drive if running in Google Colab
    from google.colab import drive
    drive.mount('/content/drive')
    df_salary_raw, df_people_raw, df_descriptions_raw = load_data(data_path="/content/drive/MyDrive/Postgrado ciencia de datos/pwc-challenge/data/")
else:
    # Load data from local directory
    df_salary_raw, df_people_raw, df_descriptions_raw = load_data(data_path="../data/")

In [646]:
df_salary_people_raw = df_salary_raw.join(df_people_raw,
                                      on="id", how="inner")
df_salary_people_raw

id,Salary,Age,Gender,Education Level,Job Title,Years of Experience
i64,f64,f64,str,str,str,f64
0,90000.0,32.0,"""Male""","""Bachelor's""","""Software Engineer""",5.0
1,65000.0,28.0,"""Female""","""Master's""","""Data Analyst""",3.0
2,150000.0,45.0,"""Male""","""PhD""","""Senior Manager""",15.0
3,60000.0,36.0,"""Female""","""Bachelor's""","""Sales Associate""",7.0
4,200000.0,52.0,"""Male""","""Master's""","""Director""",20.0
5,55000.0,29.0,"""Male""","""Bachelor's""","""Marketing Analyst""",2.0
6,120000.0,42.0,"""Female""","""Master's""","""Product Manager""",12.0
7,80000.0,31.0,"""Male""","""Bachelor's""","""Sales Manager""",4.0
8,45000.0,26.0,"""Female""","""Bachelor's""","""Marketing Coordinator""",1.0


In [647]:
fig = px.histogram(df_salary_raw,
                   x="Salary",
                   nbins=30,
                   title="Salary Distribution")
fig.show()

In [648]:
fig = px.histogram(df_salary_people_raw,
                   x="Age",
                   nbins=30,
                   title="Salary Distribution")
fig.show()

## Feature engineer

In [649]:
num_cols = ['Age']
cat_cols = ['Gender', 'Education Level', 'Job Title', 'Years of Experience']

### Target variable treatment

In [650]:
def clean_target_variable(df, target_col=TARGET):
    """Remove rows with null values in target variable."""
    initial_count = df.shape[0]
    df_cleaned = df.filter(pl.col(target_col).is_not_null())
    removed_count = initial_count - df_cleaned.shape[0]
    
    if removed_count > 0:
        print(f"Removed {removed_count} rows with null {target_col} values")
    
    return df_cleaned

### Age treatment

In [651]:
df_salary_people_raw['Age'].describe()

statistic,value
str,f64
"""count""",370.0
"""null_count""",5.0
"""mean""",37.437838
"""std""",7.080465
"""min""",23.0
"""25%""",31.0
"""50%""",36.0
"""75%""",44.0
"""max""",53.0


In [652]:
def engineer_age_features(df):
    """Engineer age-related features."""
    
    # Create age bins
    df = df.with_columns(
        pl.when(pl.col('Age').is_not_null()).then(
            pl.when(pl.col('Age') < 30).then(pl.lit('young'))
            .when(pl.col('Age') < 45).then(pl.lit('mid'))
            .otherwise(pl.lit('old')))
            .otherwise(None).alias('Age_bin')
    )

    cat_cols.append('Age_bin')
    
    return df

### Gender treatment

In [653]:
def engineer_gender_features(df):
    """Engineer gender-related features."""
    
    pass
    
    return df

### Education treatment

In [654]:
def engineer_ed_features(df):
    """engineer education-related features."""

    pass
    
    return df

### Job title treatment

In [655]:
# df_salary_people_raw['Job Title'].value_counts().sort('count', descending=True)

In [656]:
# df_salary_people_raw.filter(pl.col('Job Title').str.contains('Director'))

In [657]:
# df_salary_people_raw.filter(pl.col('Job Title').str.contains('Manager')).group_by('Job Title').agg([
#     pl.col('Job Title').count().alias('count'),
#     pl.col('Salary').mean().alias('mean_salary')
# ]).sort('mean_salary', descending=True)

In [658]:

# df_salary_people_raw.filter(~pl.col('Job Title').str.contains('Coordinator') & ~pl.col('Job Title').str.contains('Analyst') & ~pl.col('Job Title').str.contains('Manager') & ~pl.col('Job Title').str.contains('Director')).group_by('Job Title').agg([
#     pl.col('Job Title').count().alias('count'),
#     pl.col('Salary').mean().alias('mean_salary')
# ]).sort('mean_salary', descending=True)

In [659]:
def engineer_jt_features(df):
    """engineer job title-related features."""

    # Create a new column for job title categories
    df = df.with_columns(
        pl.when(pl.col('Job Title').str.contains('Director')).then(pl.lit('Director'))
        .when(pl.col('Job Title').str.contains('Manager')).then(pl.lit('Manager'))
        .when(pl.col('Job Title').str.contains('Coordinator')).then(pl.lit('Coordinator'))
        .when(pl.col('Job Title').str.contains('Analyst')).then(pl.lit('Analyst'))
        .when(pl.col('Job Title').str.contains('CEO')).then(pl.lit('Executive'))
        .when(pl.col('Job Title').str.contains('Chief')).then(pl.lit('Executive'))
        .when(pl.col('Job Title').str.contains('VP')).then(pl.lit('Executive'))
        .otherwise(pl.lit('Other')).alias('Job_Title_Category')
    )

    df = df.with_columns(
        pl.when(pl.col('Job Title').str.contains('Senior')).then(pl.lit('Senior'))
        .when(pl.col('Job Title').str.contains('Junior')).then(pl.lit('Junior'))
        .otherwise(pl.lit('Other')).alias('Job_Level')
    )

    df = df.drop('Job Title')
    cat_cols.remove('Job Title')

    cat_cols.append('Job_Title_Category')
    cat_cols.append('Job_Level')
    
    return df

### Years of experience treatment

In [674]:
df_salary_people_raw['Years of Experience'].describe()

statistic,value
str,f64
"""count""",373.0
"""null_count""",2.0
"""mean""",10.030831
"""std""",6.557007
"""min""",0.0
"""25%""",4.0
"""50%""",9.0
"""75%""",15.0
"""max""",25.0


In [660]:
def engineer_yoe_features(df):
    """engineer years of experience-related features."""

    # create years of experience bins
    df = df.with_columns(
        pl.when(pl.col('Years of Experience') < 1).then(pl.lit('junior'))
        .when(pl.col('Years of Experience') < 3).then(pl.lit('mid'))
        .otherwise(pl.lit('senior')).alias('Experience_bin')
    )
    cat_cols.append('Experience_bin')
    
    return df

### Add interaction features

In [661]:
def create_interaction_features(df):
    """Create interaction features"""
    ed_age = df.group_by('Education Level').agg(
        pl.col('Age').mean().alias('Education_Age_mean'),
        pl.col('Age').std().alias('Education_Age_std')
    )
    
    df = df.join(ed_age, on='Education Level', how='left')

    num_cols.extend(['Education_Age_mean', 'Education_Age_std'])
    
    return df

In [662]:
def perform_feature_engineering(df):
    """Perform all feature engineering steps."""
    # Remove ID column
    df = df.drop(ID)
    
    # Clean target variable
    df = clean_target_variable(df)
    
    # Engineer features
    df = engineer_age_features(df)
    df = engineer_gender_features(df)
    df = engineer_ed_features(df)
    df = engineer_jt_features(df)
    df = engineer_yoe_features(df)
    df = create_interaction_features(df)
    
    return df

df_salary_people = perform_feature_engineering(df_salary_people_raw)

Removed 2 rows with null Salary values


## Data preprocessing

In [663]:
# Salary has outliers, need to decide how to handle them

# age has 3 null values, need to decide how to fill them

# gender has 3 null values, need to decide how to fill them

# Education Level has 3 null values, need to decide how to fill them

# Job Title has 3 null values, need to decide how to fill them
# Job title has high cardinality, experiment extracting keywords

In [664]:
def create_preprocessing_pipeline():
    """Create sklearn preprocessing pipeline."""
    # numerical imputer
    mean_imputer = SimpleImputer(strategy='mean')
    median_imputer = SimpleImputer(strategy='median')
    # numerical scaler
    scaler = StandardScaler()
    # categorical imputer
    most_freq_imputer = SimpleImputer(strategy='most_frequent')
    unknown_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
    # categorical encoder
    oe_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    ohe_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    # numerical pipeline
    num_pipeline = Pipeline([
        ('mean_imputer', mean_imputer),
        ('scaler', scaler)
    ])
    # categorical pipeline
    cat_pipeline = Pipeline([
        ('most_freq_imputer', most_freq_imputer),
        ('oe_encoder', oe_encoder)
    ])

    # column transformer

    pipeline = ColumnTransformer([
        ('num', num_pipeline, num_cols),
        ('cat_oe', cat_pipeline, cat_cols),
        ])
    return pipeline


In [665]:
X = df_salary_people.drop(TARGET)
y = df_salary_people[TARGET]

## Metric definition

In [666]:
def metrics(y_true, y_pred):
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {'rmse': rmse, 'r2': r2}

## Cross validation training

In [667]:
def feature_importance(pipeline, model):
    """Get feature importance from the model."""
    importances = []
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importances = np.abs(model.coef_)
    else:
        importances = np.zeros(len(pipeline.get_feature_names_out()))
    
    fe_impo = pl.DataFrame({
        'feature': pipeline.get_feature_names_out(),
        'importance': importances
    }).sort('importance', descending=True)
    
    return fe_impo

In [668]:
def cross_validation(model, X, y):

    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    results = []
    importances_table = None

    for fold, (train_index, val_index) in enumerate(kf.split(X)):

        # print(f"Fold {fold + 1}/{N_FOLDS}")

        X_train, X_val = X[train_index].clone(), X[val_index].clone()
        y_train, y_val = y[train_index].clone(), y[val_index].clone()

        # Create preprocessing pipeline
        preprocessing_pipeline = create_preprocessing_pipeline()

        X_train = preprocessing_pipeline.fit_transform(X_train)
        X_val = preprocessing_pipeline.transform(X_val)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        result = metrics(y_val, y_pred)
        results.append(result)

        # Get feature importance for this fold
        fe_impo = feature_importance(preprocessing_pipeline, model)

        
        
        # Rename the importance column to include fold number
        fe_impo = fe_impo.with_columns(
            pl.col('importance').alias(f'fold_{fold + 1}')
        ).drop('importance')
        
        # Initialize or join the importances table
        if importances_table is None:
            # First fold - create the base table
            importances_table = fe_impo
        else:
            # Subsequent folds - join on feature names
            importances_table = importances_table.join(
                fe_impo, 
                on='feature', 
                how='inner'
            )

        print(f"Fold {fold + 1} - RMSE: {result['rmse']:.2f}, R2: {result['r2']:.2f}")

    
    # Add summary statistics across folds
    fold_columns = [f'fold_{i+1}' for i in range(N_FOLDS)]
    
    # Calculate mean and std across only the fold columns (numeric)
    importances_table = importances_table.with_columns([
        pl.mean_horizontal([pl.col(col).round(2) for col in fold_columns]).alias('mean_importance'),
        pl.concat_list([pl.col(col).round(2) for col in fold_columns]).list.std().alias('std_importance')
    ])
    # Round numeric columns to 2 decimals
    numeric_cols = fold_columns + ['mean_importance', 'std_importance']
    importances_table = importances_table.with_columns([
        pl.col(col).round(2) for col in numeric_cols
    ])

    # Sort by mean importance
    importances_table = importances_table.sort('mean_importance', descending=True)

    # print mean metrics across folds
    avg_rmse = np.mean([result['rmse'] for result in results])
    avg_r2 = np.mean([result['r2'] for result in results])
    print(f"\nAverage RMSE across folds: {avg_rmse:.2f}")
    print(f"Average R2 across folds: {avg_r2:.2f}")
    
    return results, importances_table

## Model evaluation

In [669]:
model_dummy = DummyRegressor(strategy='mean')
results_dummy, _ = cross_validation(model_dummy, X, y)

Fold 1 - RMSE: 55377.16, R2: -0.03
Fold 2 - RMSE: 44249.10, R2: -0.00
Fold 3 - RMSE: 44752.30, R2: -0.00
Fold 4 - RMSE: 48495.19, R2: -0.00
Fold 5 - RMSE: 47991.51, R2: -0.05

Average RMSE across folds: 48173.05
Average R2 across folds: -0.02


In [670]:
# linear regression model
model_lr = LinearRegression()
results_lr, importances_lr = cross_validation(model_lr, X, y)
importances_lr

Fold 1 - RMSE: 19181.20, R2: 0.88
Fold 2 - RMSE: 13020.96, R2: 0.91
Fold 3 - RMSE: 12262.68, R2: 0.92
Fold 4 - RMSE: 23813.81, R2: 0.76
Fold 5 - RMSE: 13540.41, R2: 0.92

Average RMSE across folds: 16363.81
Average R2 across folds: 0.88


feature,fold_1,fold_2,fold_3,fold_4,fold_5,mean_importance,std_importance
str,f64,f64,f64,f64,f64,f64,f64
"""num__Age""",13424.9,19147.23,17036.23,14438.39,19261.04,16661.56,2669.02
"""num__Education_Age_mean""",9758.09,8890.03,9230.25,8143.94,11712.15,9546.89,1344.71
"""cat_oe__Gender""",8571.06,9452.9,8416.35,6600.82,7955.21,8199.27,1045.41
"""cat_oe__Job_Level""",7052.99,6076.42,5331.09,5674.09,5677.16,5962.35,664.35
"""cat_oe__Years of Experience""",3213.57,2586.93,3141.28,3652.89,2737.52,3066.44,421.1
"""cat_oe__Education Level""",1846.18,3658.98,1643.92,1076.41,2068.73,2058.84,967.35
"""num__Education_Age_std""",983.58,2220.76,1276.52,2392.6,1062.53,1587.2,668.25
"""cat_oe__Age_bin""",1280.75,2435.92,832.5,636.08,1836.71,1404.39,738.85
"""cat_oe__Experience_bin""",1045.15,2067.76,431.93,808.93,474.93,965.74,665.43


In [671]:
# lasso regression model
model_lasso = Lasso(alpha=50, random_state=SEED)
results_lasso, importances_lasso = cross_validation(model_lasso, X, y)
importances_lasso

Fold 1 - RMSE: 19193.67, R2: 0.88
Fold 2 - RMSE: 12855.99, R2: 0.92
Fold 3 - RMSE: 12287.91, R2: 0.92
Fold 4 - RMSE: 24354.86, R2: 0.75
Fold 5 - RMSE: 13393.89, R2: 0.92

Average RMSE across folds: 16417.27
Average R2 across folds: 0.88


feature,fold_1,fold_2,fold_3,fold_4,fold_5,mean_importance,std_importance
str,f64,f64,f64,f64,f64,f64,f64
"""num__Age""",12362.59,17952.73,15920.55,13019.05,18074.11,15465.81,2683.81
"""num__Education_Age_mean""",9904.28,9158.02,9363.97,8270.4,10158.27,9370.99,735.0
"""cat_oe__Gender""",8292.39,9217.83,8159.39,6314.61,7731.1,7943.06,1059.56
"""cat_oe__Job_Level""",6901.78,5932.56,5174.55,5483.78,5562.29,5810.99,666.82
"""cat_oe__Years of Experience""",3410.26,2802.87,3330.62,3858.79,2914.87,3263.48,422.53
"""num__Education_Age_std""",869.61,2084.59,1171.14,2231.84,1166.04,1504.64,611.21
"""cat_oe__Education Level""",1471.3,3099.84,1273.55,690.82,0.0,1307.1,1154.37
"""cat_oe__Age_bin""",841.17,1883.3,537.84,663.43,1829.68,1151.08,653.18
"""cat_oe__Job_Title_Category""",688.21,733.12,407.35,739.52,816.55,676.95,157.61


In [672]:
rfr = RandomForestRegressor(random_state=SEED)
results_rf, importances_rf = cross_validation(rfr, X, y)
importances_rf

Fold 1 - RMSE: 12973.57, R2: 0.94
Fold 2 - RMSE: 9181.73, R2: 0.96
Fold 3 - RMSE: 10499.40, R2: 0.94
Fold 4 - RMSE: 21086.94, R2: 0.81
Fold 5 - RMSE: 10239.98, R2: 0.95

Average RMSE across folds: 12796.33
Average R2 across folds: 0.92


feature,fold_1,fold_2,fold_3,fold_4,fold_5,mean_importance,std_importance
str,f64,f64,f64,f64,f64,f64,f64
"""num__Age""",0.43,0.42,0.53,0.32,0.5,0.44,0.08
"""cat_oe__Years of Experience""",0.45,0.45,0.34,0.59,0.37,0.44,0.1
"""cat_oe__Job_Level""",0.04,0.04,0.05,0.02,0.05,0.04,0.01
"""cat_oe__Job_Title_Category""",0.03,0.05,0.04,0.03,0.04,0.04,0.01
"""num__Education_Age_mean""",0.02,0.02,0.02,0.01,0.02,0.02,0.0
"""cat_oe__Education Level""",0.01,0.01,0.01,0.01,0.01,0.01,0.0
"""num__Education_Age_std""",0.01,0.01,0.0,0.01,0.01,0.01,0.0
"""cat_oe__Gender""",0.01,0.0,0.01,0.01,0.0,0.01,0.01
"""cat_oe__Experience_bin""",0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [673]:
lgbm = LGBMRegressor(random_state=SEED, verbosity=-1)
results_lgb, importances_lgbm = cross_validation(lgbm, X, y)
importances_lgbm

Fold 1 - RMSE: 15658.67, R2: 0.92
Fold 2 - RMSE: 9319.91, R2: 0.96
Fold 3 - RMSE: 9142.96, R2: 0.96
Fold 4 - RMSE: 20710.83, R2: 0.82
Fold 5 - RMSE: 9483.89, R2: 0.96

Average RMSE across folds: 12863.25
Average R2 across folds: 0.92


feature,fold_1,fold_2,fold_3,fold_4,fold_5,mean_importance,std_importance
str,i32,i32,i32,i32,i32,f64,f64
"""num__Age""",261,327,318,314,315,307.0,26.22
"""cat_oe__Years of Experience""",284,243,271,225,252,255.0,23.18
"""cat_oe__Job_Title_Category""",149,175,191,209,179,180.6,22.06
"""cat_oe__Job_Level""",112,103,91,95,97,99.6,8.17
"""cat_oe__Gender""",72,78,75,109,69,80.6,16.23
"""num__Education_Age_mean""",111,81,63,63,65,76.6,20.66
"""num__Education_Age_std""",15,12,7,6,9,9.8,3.7
"""cat_oe__Age_bin""",12,1,9,4,9,7.0,4.42
"""cat_oe__Education Level""",1,6,0,0,1,1.6,2.51
